In [5]:
import pandas as pd
from matplotlib import pyplot as plt

df = pd.read_csv("churn-7043.csv")

In [6]:
df=df.drop(['customerID'], axis=1) 
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)


##DISCRITIZAION 
for col in num_df.columns:
    labels =[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    cats=pd.cut(num_df[col], 20, labels=labels)
    num_df[col]=cats
    
##DISCRITIZAION END  

result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)
 

result_df_op=result_df

X=result_df_op
y=churn
 

In [7]:
X

,SeniorCitizen,tenure,MonthlyCharges,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges
0,1,1,3,0,1,0,0,1,0,0,2,0,0,0,0,0,1,2,2505
1,1,10,8,1,0,0,1,0,0,2,0,2,0,0,0,1,0,3,1466
2,1,1,8,1,0,0,1,0,0,2,2,0,0,0,0,0,1,3,157
3,1,13,5,1,0,0,0,1,0,2,0,2,2,0,0,1,0,0,1400
4,1,1,11,0,0,0,1,0,1,0,0,0,0,0,0,0,1,2,925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,1,7,14,1,1,1,1,2,0,2,0,2,2,2,2,1,1,3,1597
7039,1,20,17,0,1,1,1,2,1,0,2,2,0,2,2,1,1,1,5698
7040,1,4,3,0,1,1,0,1,0,2,0,0,0,0,0,0,1,2,2994
7041,20,2,12,1,1,0,1,2,1,0,0,0,0,0,0,0,1,3,2660


In [8]:
X.shape

(7043, 19)

In [9]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
1,tenure,4187.962477
0,SeniorCitizen,1927.143682
18,TotalCharges,1616.483021
15,Contract,1115.780167
2,MonthlyCharges,961.807817
9,OnlineSecurity,551.611529
12,TechSupport,523.303866
10,OnlineBackup,230.086520
11,DeviceProtection,191.303140
5,Dependents,133.036443


In [10]:
y_train=y
x_train_chi = select_feature.transform(X)
x_train_chi.shape

(7043, 15)

In [11]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

3955 1219 493 1376
pod:  0.7362225789192082
pof:  0.2356010823347507
AUC:  0.7503107482922288


In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits
{'var_smoothing': 1.232846739442066e-06}
GaussianNB(var_smoothing=1.232846739442066e-06)
0.7832364520829301


In [13]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=3.5111917342151277e-07)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=10, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

4173 1001 614 1255
pod:  0.671482075976458
pof:  0.1934673366834171
AUC:  0.7390073696465205
accuracy:  0.7706943064035212


In [14]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
{'C': 6, 'multi_class': 'ovr', 'penalty': 'l1', 'solver': 'liblinear'}
LogisticRegression(C=6, multi_class='ovr', penalty='l1', solver='liblinear')
0.7989476547388781


In [16]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', verbose=0,
                   warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4643 531 888 981
pod:  0.5248796147672552
pof:  0.10262852725164283
AUC:  0.7111255437578062
accuracy:  0.7985233565242085


In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 10 folds for each of 16 candidates, totalling 160 fits
{'metric': 'manhattan', 'n_neighbors': 19, 'weights': 'uniform'}
KNeighborsClassifier(metric='manhattan', n_neighbors=19)
0.7587693423597679


In [18]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(metric='manhattan', weights='distance', n_neighbors=19 )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4827 347 1355 514
pod:  0.27501337613697163
pof:  0.06706609972941631
AUC:  0.6039736382037777
accuracy:  0.7583416157887264


In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 288 candidates, totalling 2880 fits
{'bootstrap': True, 'max_depth': 100, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 12, 'n_estimators': 300}
RandomForestClassifier(max_depth=100, max_features=3, min_samples_leaf=3,
                       min_samples_split=12, n_estimators=300)
0.8046256447453256


In [20]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(max_depth=80, max_features=3, min_samples_leaf=5,
                       min_samples_split=8, n_estimators=300)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4726 448 956 913
pod:  0.4884965222043874
pof:  0.08658678005411674
AUC:  0.7009548710751353
accuracy:  0.8006531307681386


In [21]:
# Decision Tree
import time
start = time.time()
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 


#param_grid={
#    'criterion': ['gini', 'entropy'],
#    'splitter': ['best', 'random'],
#    'min_samples_split' : range(10,500,20),
#    'max_depth': [1, 2, 3, 4, 5, 20],
#    'max_features': [1, 2, 3, 4],
#    'max_leaf_nodes': list(range(2, 100)), 
#    'min_samples_split': [2, 3, 4]
#} 

param_grid={
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
} 

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = param_grid, 
                          cv = 10, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)
end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
{'criterion': 'entropy', 'splitter': 'random'}
DecisionTreeClassifier(criterion='entropy', splitter='random')
0.7330663684719536
Time:  0.19768357276916504


In [22]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
classifier =DecisionTreeClassifier(criterion='entropy', splitter='random')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4226 948 931 938
pod:  0.50187265917603
pof:  0.18322381136451488
AUC:  0.6593244239057575
accuracy:  0.7332102797103507


In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

param_grid = {
    "loss":["deviance"],
    "learning_rate": [0.01, 0.025, 0.05, 0.075, 0.1, 0.15, 0.2],
    "min_samples_split": np.linspace(0.1, 0.5, 12),
    "min_samples_leaf": np.linspace(0.1, 0.5, 12),
    "max_depth":[3,5,8],
    "max_features":["log2","sqrt"],
    "criterion": ["friedman_mse",  "mae"],
    "subsample":[0.5, 0.618, 0.8, 0.85, 0.9, 0.95, 1.0],
    "n_estimators":[10]
    }

grid = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=10, n_jobs=-1)

grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

{'criterion': 'friedman_mse', 'learning_rate': 0.2, 'loss': 'deviance', 'max_depth': 3, 'max_features': 'log2', 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'n_estimators': 10, 'subsample': 1.0}
GradientBoostingClassifier(learning_rate=0.2, max_features='log2',
                           min_samples_leaf=0.1, min_samples_split=0.1,
                           n_estimators=10)
0.7936964055448097


In [25]:
#  Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
classifier =GradientBoostingClassifier(learning_rate=0.2, max_features='log2',
                           min_samples_leaf=0.1, min_samples_split=0.1,
                           n_estimators=10)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4896 278 1214 655
pod:  0.35045478865703583
pof:  0.05373018940858137
AUC:  0.6483622996242272
accuracy:  0.7881584552037484


In [26]:
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    n_cols = x_train_chi.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        n_cols = x_train_chi.shape[1]
        nn = Sequential()
        nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        nn.add(Dense(250, activation='relu'))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        return nn
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

Best: 0.788159 using {'batch_size': 20, 'epochs': 100}
0.763877 (0.024528) with: {'batch_size': 10, 'epochs': 10}
0.771549 (0.018770) with: {'batch_size': 10, 'epochs': 50}
0.767570 (0.022783) with: {'batch_size': 10, 'epochs': 100}
0.694882 (0.073958) with: {'batch_size': 20, 'epochs': 10}
0.773672 (0.019636) with: {'batch_size': 20, 'epochs': 50}
0.788159 (0.006611) with: {'batch_size': 20, 'epochs': 100}
0.728669 (0.025193) with: {'batch_size': 40, 'epochs': 10}
0.739173 (0.039539) with: {'batch_size': 40, 'epochs': 50}
0.759482 (0.032189) with: {'batch_size': 40, 'epochs': 100}
0.497076 (0.177494) with: {'batch_size': 60, 'epochs': 10}
0.780634 (0.014267) with: {'batch_size': 60, 'epochs': 50}
0.753227 (0.014261) with: {'batch_size': 60, 'epochs': 100}
0.737185 (0.004611) with: {'batch_size': 80, 'epochs': 10}
0.743012 (0.029582) with: {'batch_size': 80, 'epochs': 50}
0.779500 (0.022670) with: {'batch_size': 80, 'epochs': 100}
0.640650 (0.073877) with: {'batch_size': 100, 'epochs':

In [27]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

import time
start = time.time()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2,  epochs= 100, batch_size= 20, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix


from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/100
254/254 [==============================] - 2s 5ms/step - loss: 11.8162 - accuracy: 0.6014 - val_loss: 1.5952 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 2ms/step - loss: 1.2373 - accuracy: 0.6451 - val_loss: 0.6277 - val_accuracy: 0.6112
Epoch 3/100
254/254 [==============================] - 1s 2ms/step - loss: 0.5886 - accuracy: 0.7206 - val_loss: 0.5300 - val_accuracy: 0.7610
Epoch 4/100
254/254 [==============================] - 1s 2ms/step - loss: 0.5239 - accuracy: 0.7346 - val_loss: 0.4859 - val_accuracy: 0.7595
Epoch 5/100
254/254 [==============================] - 1s 2ms/step - loss: 0.5657 - accuracy: 0.7254 - val_loss: 0.4828 - val_accuracy: 0.7429
Epoch 6/100
254/254 [==============================] - 1s 2ms/step - loss: 0.5248 - accuracy: 0.7405 - val_loss: 0.5084 - val_accuracy: 0.7287
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5062 - accuracy: 0.7445 - val_loss: 0.4643 - val_accuracy: 0.728

254/254 [==============================] - 2s 4ms/step - loss: 13.9674 - accuracy: 0.6130 - val_loss: 0.8098 - val_accuracy: 0.7311
Epoch 2/100
254/254 [==============================] - 1s 2ms/step - loss: 1.0465 - accuracy: 0.6335 - val_loss: 0.7799 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 2ms/step - loss: 0.6908 - accuracy: 0.6963 - val_loss: 0.6192 - val_accuracy: 0.7374
Epoch 4/100
254/254 [==============================] - 1s 2ms/step - loss: 0.6689 - accuracy: 0.7193 - val_loss: 0.5805 - val_accuracy: 0.7287
Epoch 5/100
254/254 [==============================] - 1s 2ms/step - loss: 0.6044 - accuracy: 0.7178 - val_loss: 0.7081 - val_accuracy: 0.5024
Epoch 6/100
254/254 [==============================] - 1s 2ms/step - loss: 0.6338 - accuracy: 0.7033 - val_loss: 0.5953 - val_accuracy: 0.7279
Epoch 7/100
254/254 [==============================] - 1s 2ms/step - loss: 0.5987 - accuracy: 0.7162 - val_loss: 0.5799 - val_accuracy: 0.7287
Epoch 8/10

254/254 [==============================] - 2s 4ms/step - loss: 10.5217 - accuracy: 0.6208 - val_loss: 2.1886 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.0573 - accuracy: 0.6547 - val_loss: 0.7942 - val_accuracy: 0.7271
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7570 - accuracy: 0.6917 - val_loss: 0.6327 - val_accuracy: 0.6443
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5305 - accuracy: 0.7372 - val_loss: 0.6914 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5282 - accuracy: 0.7429 - val_loss: 0.4821 - val_accuracy: 0.7555
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5037 - accuracy: 0.7535 - val_loss: 0.4727 - val_accuracy: 0.7603
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.4936 - accuracy: 0.7454 - val_loss: 0.4687 - val_accuracy: 0.7626
Epoch 8/10

254/254 [==============================] - 2s 4ms/step - loss: 9.8765 - accuracy: 0.6283 - val_loss: 1.6550 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.9872 - accuracy: 0.6590 - val_loss: 0.5868 - val_accuracy: 0.7319
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7281 - accuracy: 0.7046 - val_loss: 0.5075 - val_accuracy: 0.7492
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5522 - accuracy: 0.7467 - val_loss: 0.6037 - val_accuracy: 0.7326
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5628 - accuracy: 0.7378 - val_loss: 0.6888 - val_accuracy: 0.7319
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5396 - accuracy: 0.7384 - val_loss: 0.6263 - val_accuracy: 0.6183
y2_pred:  [[5.93272567e-01]
 [7.19552755e-01]
 [3.09320837e-01]
 [4.33639199e-01]
 [5.63876629e-01]
 [1.78889930e-02]
 [4.70419496e-01]
 [7.69202709e-01]


254/254 [==============================] - 2s 4ms/step - loss: 11.9424 - accuracy: 0.6116 - val_loss: 1.1630 - val_accuracy: 0.7295
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.0905 - accuracy: 0.6485 - val_loss: 0.5741 - val_accuracy: 0.7287
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6841 - accuracy: 0.7141 - val_loss: 0.9154 - val_accuracy: 0.4243
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6636 - accuracy: 0.6802 - val_loss: 0.5326 - val_accuracy: 0.7342
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5959 - accuracy: 0.7237 - val_loss: 0.5550 - val_accuracy: 0.7295
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5604 - accuracy: 0.7216 - val_loss: 0.6698 - val_accuracy: 0.5213
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5608 - accuracy: 0.7157 - val_loss: 0.4962 - val_accuracy: 0.7326
Epoch 8/10

254/254 [==============================] - 2s 4ms/step - loss: 12.1988 - accuracy: 0.6073 - val_loss: 1.5274 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.2362 - accuracy: 0.6451 - val_loss: 0.5423 - val_accuracy: 0.7287
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6519 - accuracy: 0.7066 - val_loss: 0.6362 - val_accuracy: 0.6420
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5945 - accuracy: 0.7252 - val_loss: 0.5090 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5590 - accuracy: 0.7239 - val_loss: 0.4946 - val_accuracy: 0.7429
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5792 - accuracy: 0.7261 - val_loss: 0.5636 - val_accuracy: 0.7303
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5647 - accuracy: 0.7415 - val_loss: 0.5519 - val_accuracy: 0.7279
Epoch 8/10

254/254 [==============================] - 2s 4ms/step - loss: 14.0605 - accuracy: 0.6096 - val_loss: 2.9351 - val_accuracy: 0.7287
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.9686 - accuracy: 0.6601 - val_loss: 0.5352 - val_accuracy: 0.7366
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6784 - accuracy: 0.7247 - val_loss: 0.5015 - val_accuracy: 0.7405
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5662 - accuracy: 0.7438 - val_loss: 0.4995 - val_accuracy: 0.7437
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5282 - accuracy: 0.7433 - val_loss: 0.4942 - val_accuracy: 0.7445
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5082 - accuracy: 0.7410 - val_loss: 1.0167 - val_accuracy: 0.4692
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5158 - accuracy: 0.7469 - val_loss: 0.5061 - val_accuracy: 0.7366
Epoch 8/10

254/254 [==============================] - 2s 4ms/step - loss: 12.7220 - accuracy: 0.5971 - val_loss: 1.4779 - val_accuracy: 0.7279
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.8310 - accuracy: 0.6495 - val_loss: 0.5795 - val_accuracy: 0.7279
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6655 - accuracy: 0.7127 - val_loss: 0.6505 - val_accuracy: 0.6585
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6639 - accuracy: 0.6968 - val_loss: 0.6507 - val_accuracy: 0.7279
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6019 - accuracy: 0.7268 - val_loss: 0.6061 - val_accuracy: 0.7279
y2_pred:  [[2.91465104e-01]
 [1.93624496e-01]
 [1.51404113e-01]
 [1.77364677e-01]
 [1.67105287e-01]
 [1.25793278e-01]
 [3.00206542e-01]
 [2.34492779e-01]
 [9.10002291e-02]
 [2.39623696e-01]
 [3.14434111e-01]
 [2.09317446e-01]
 [2.11167127e-01]
 [2.46340156e-01]
 [1.42816395e-01]
 [2.08606362e-01

254/254 [==============================] - 2s 5ms/step - loss: 7.0307 - accuracy: 0.6110 - val_loss: 0.7965 - val_accuracy: 0.7248
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7307 - accuracy: 0.6976 - val_loss: 0.7487 - val_accuracy: 0.7248
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7030 - accuracy: 0.6959 - val_loss: 0.8778 - val_accuracy: 0.4109
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6483 - accuracy: 0.6935 - val_loss: 0.5809 - val_accuracy: 0.7248
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5943 - accuracy: 0.7188 - val_loss: 0.7089 - val_accuracy: 0.7248
Epoch 6/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6220 - accuracy: 0.7173 - val_loss: 0.5924 - val_accuracy: 0.7248
Epoch 7/100
254/254 [==============================] - 1s 3ms/step - loss: 0.5723 - accuracy: 0.7364 - val_loss: 0.5607 - val_accuracy: 0.7263
Epoch 8/100

254/254 [==============================] - 2s 5ms/step - loss: 19.4233 - accuracy: 0.6049 - val_loss: 1.1865 - val_accuracy: 0.3289
Epoch 2/100
254/254 [==============================] - 1s 3ms/step - loss: 1.0747 - accuracy: 0.6413 - val_loss: 0.5677 - val_accuracy: 0.7248
Epoch 3/100
254/254 [==============================] - 1s 3ms/step - loss: 0.7350 - accuracy: 0.7005 - val_loss: 0.9494 - val_accuracy: 0.3864
Epoch 4/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6921 - accuracy: 0.6831 - val_loss: 0.5717 - val_accuracy: 0.7248
Epoch 5/100
254/254 [==============================] - 1s 3ms/step - loss: 0.6100 - accuracy: 0.7159 - val_loss: 0.6944 - val_accuracy: 0.7248
y2_pred:  [[0.19176531]
 [0.15354484]
 [0.02764592]
 [0.02823752]
 [0.03693283]
 [0.0801236 ]
 [0.04852465]
 [0.2241571 ]
 [0.18709525]
 [0.05856517]
 [0.0375874 ]
 [0.37148598]
 [0.2834849 ]
 [0.21577877]
 [0.03698611]
 [0.04881141]
 [0.08720416]
 [0.07075483]
 [0.04131997]
 [0.02657571]
 [0.29

In [ ]:
##RNN Hyperparemeter tuning
import time
start = time.time()

# Use scikit-learn to grid search the batch size and epochs
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
 
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense,Dropout, LSTM, GRU
from keras.layers import Embedding
# Function to create model, required for KerasClassifier
def create_model(): 
    #model = Sequential()
    #model.add(Dense(12, input_shape=(8,), activation='relu'))
    #model.add(Dense(1, activation='sigmoid'))
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model = Sequential()
    #get number of columns in training data
    #n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model
def nn_cl_fun():
        #opt = Adam(lr = learning_rate)
        #n_cols = X2_train.shape[1]
        #nn = Sequential()
        #nn.add(Dense(10, input_dim=n_cols, activation='relu'))
        #nn.add(Dense(250, activation='relu'))
        #nn.add(Dense(1, activation='sigmoid'))
        #nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        model = Sequential()
        model.add(Embedding(10000, 32))
        model.add(LSTM(32))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
        return model
# fix random seed for reproducibility
seed = 7
tf.random.set_seed(seed)

# create model
#model = KerasClassifier(model=create_model, verbose=0)
model = KerasClassifier(build_fn=nn_cl_fun, verbose=0)

# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(x_train_chi, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

end = time.time()
time_taken = end - start
print('Time: ',time_taken)  

In [28]:
#RNN
import time
start = time.time()

def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=10, batch_size=100, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)

end = time.time()
time_taken = end - start
print('Time: ',time_taken)

Epoch 1/10
51/51 [==============================] - 7s 27ms/step - loss: 0.6162 - acc: 0.6978 - val_loss: 0.5343 - val_acc: 0.7279
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5217 - acc: 0.7304 - val_loss: 0.5212 - val_acc: 0.7303
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4938 - acc: 0.7425 - val_loss: 0.5140 - val_acc: 0.7279
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4959 - acc: 0.7442 - val_loss: 0.5154 - val_acc: 0.7358
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4795 - acc: 0.7559 - val_loss: 0.5098 - val_acc: 0.7311
Epoch 6/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4643 - acc: 0.7775 - val_loss: 0.5054 - val_acc: 0.7342
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4396 - acc: 0.7902 - val_loss: 0.5066 - val_acc: 0.7405
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4079 - a

Epoch 1/10
51/51 [==============================] - 3s 26ms/step - loss: 0.6097 - acc: 0.7334 - val_loss: 0.5389 - val_acc: 0.7279
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5182 - acc: 0.7322 - val_loss: 0.5163 - val_acc: 0.7342
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5011 - acc: 0.7394 - val_loss: 0.5130 - val_acc: 0.7295
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4919 - acc: 0.7477 - val_loss: 0.5164 - val_acc: 0.7350
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4813 - acc: 0.7562 - val_loss: 0.5115 - val_acc: 0.7342
Epoch 6/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4541 - acc: 0.7798 - val_loss: 0.5094 - val_acc: 0.7405
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4344 - acc: 0.7922 - val_loss: 0.5057 - val_acc: 0.7461
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.3895 - a

Epoch 1/10
51/51 [==============================] - 3s 27ms/step - loss: 0.6027 - acc: 0.7293 - val_loss: 0.5272 - val_acc: 0.7287
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5210 - acc: 0.7280 - val_loss: 0.5173 - val_acc: 0.7342
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4949 - acc: 0.7459 - val_loss: 0.5127 - val_acc: 0.7303
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4893 - acc: 0.7482 - val_loss: 0.5087 - val_acc: 0.7319
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4876 - acc: 0.7490 - val_loss: 0.5066 - val_acc: 0.7374
Epoch 6/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4629 - acc: 0.7728 - val_loss: 0.5159 - val_acc: 0.7437
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4374 - acc: 0.8004 - val_loss: 0.5105 - val_acc: 0.7397
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4099 - a

Epoch 1/10
51/51 [==============================] - 3s 27ms/step - loss: 0.6026 - acc: 0.6977 - val_loss: 0.5286 - val_acc: 0.7279
Epoch 2/10
51/51 [==============================] - 1s 16ms/step - loss: 0.5196 - acc: 0.7288 - val_loss: 0.5239 - val_acc: 0.7319
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4872 - acc: 0.7482 - val_loss: 0.5153 - val_acc: 0.7279
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4771 - acc: 0.7579 - val_loss: 0.5099 - val_acc: 0.7295
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4812 - acc: 0.7551 - val_loss: 0.5063 - val_acc: 0.7358
Epoch 6/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4509 - acc: 0.7800 - val_loss: 0.5037 - val_acc: 0.7397
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4199 - acc: 0.8126 - val_loss: 0.5107 - val_acc: 0.7382
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.3809 - a

Epoch 1/10
51/51 [==============================] - 3s 27ms/step - loss: 0.6021 - acc: 0.7279 - val_loss: 0.5325 - val_acc: 0.7248
Epoch 2/10
51/51 [==============================] - 1s 17ms/step - loss: 0.5096 - acc: 0.7322 - val_loss: 0.5242 - val_acc: 0.7224
Epoch 3/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4818 - acc: 0.7530 - val_loss: 0.5122 - val_acc: 0.7248
Epoch 4/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4766 - acc: 0.7566 - val_loss: 0.5178 - val_acc: 0.7248
Epoch 5/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4791 - acc: 0.7541 - val_loss: 0.5090 - val_acc: 0.7326
Epoch 6/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4410 - acc: 0.7845 - val_loss: 0.5073 - val_acc: 0.7492
Epoch 7/10
51/51 [==============================] - 1s 16ms/step - loss: 0.4184 - acc: 0.8190 - val_loss: 0.4998 - val_acc: 0.7484
Epoch 8/10
51/51 [==============================] - 1s 16ms/step - loss: 0.3701 - a